In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd drive/MyDrive

/content/drive/MyDrive


In [3]:
!pip install transformers
!pip install sentencepiece
!pip install bertopic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 51.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 90.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.4/143.4 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 58.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 9.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 8.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 69.1 MB/s 

In [4]:
!pip install bertopic[visualization]

In [5]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-7u6xdgqr/kobert-tokenizer_2e28b35b99a346149aca18ed31840876
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-7u6xdgqr/kobert-tokenizer_2e28b35b99a346149aca18ed31840876
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4632 sha256=55fba99edcdd5774ccdf1b72cb374b79422caadac204f10bb8fdee3dbb7571f4
  Stored in directory: /tmp/pip-ephem-wheel-cache-lu4smt34/wheels/e9/1a/3f/a864970e8a169c176befa3c4a1e07aa612f69195907a4045fe
Successfully built kobert_tokenizer


In [6]:
from kobert_tokenizer import KoBERTTokenizer
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
tokenizer.encode("한국어 모델을 공유합니다.")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


[2, 4958, 6855, 2046, 7088, 1050, 7843, 54, 3]

In [7]:
import torch
from transformers import BertModel
model = BertModel.from_pretrained('skt/kobert-base-v1')
text = "한국어 모델을 공유합니다."
inputs = tokenizer.batch_encode_plus([text])
out = model(input_ids = torch.tensor(inputs['input_ids']), attention_mask = torch.tensor(inputs['attention_mask']))
out.pooler_output.shape

torch.Size([1, 768])

In [8]:
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

In [9]:
inputs = tokenizer.batch_encode_plus([text], return_tensors='pt')
model_output = model(**inputs)
mean_pooling(model_output, inputs['attention_mask'])

tensor([[ 3.6122e-02, -1.2337e-01, -7.5299e-03, -5.0620e+00,  1.6959e-01,
          9.2553e-02, -2.8838e-01,  2.1198e-01, -6.3860e-01, -3.6451e-01,
         -1.6556e-02,  1.8412e-01, -2.9757e-01,  5.1234e-02,  4.4390e-02,
          2.3413e-01,  3.8918e-01, -2.7811e-02,  2.5196e-01, -1.6740e-01,
         -5.0612e-02, -1.1393e-01, -5.8496e-02, -9.2304e-02, -3.3941e-01,
          2.9963e-02, -6.9979e-02, -1.2118e-01, -1.7573e-01,  2.4531e-01,
          1.6764e-01,  6.0146e-02, -1.3626e-01,  1.6598e-01,  1.2201e-01,
          5.2110e-02, -3.8716e-02, -3.4096e-02,  2.7250e-01, -6.6029e-02,
          2.2798e-01,  2.6542e-02,  3.8524e-01, -5.3627e-02,  9.2007e-02,
          1.2872e-01, -9.4732e-02,  1.4018e-01,  4.4361e-02,  3.0786e-02,
         -2.7000e-01,  2.5637e-02,  5.1933e-02,  1.9702e-01, -1.4610e-01,
         -2.9428e-01,  1.8972e-01, -1.1122e-01, -2.5106e-01,  1.7953e-01,
          1.1555e-01, -1.8627e-01,  1.9639e-02, -1.5012e-01,  6.5611e-02,
          6.9550e-02,  1.3693e-01,  3.

In [10]:
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer
from bertopic import BERTopic
import os
import json

In [11]:
import re

cond = re.compile('[가-힣]+')

def get_data(gallery):
  result = []
  for name in os.listdir(path='/content/drive/MyDrive/jolnon/' + gallery):
    with open('/content/drive/MyDrive/jolnon/' + gallery + '/' + name, 'r', encoding='utf-8') as f:
      data = json.load(f)
      valid = []
      for d in data['content'].split('\n'):
        res = []
        for word in d.split(' '):
          if cond.match(word) != None:
            res.append(word)
        d = ' '.join(res)
        if d and not d.replace(' ', '').isdecimal():
          valid.append(d)
      result.append('\n'.join([data['title']] + valid))
  return result

In [37]:
import random
random.seed(42)

dataset = [get_data('cock_tail'), get_data('brandy'), get_data('ulisul'), get_data('wine')]
cocktail = []
for data in dataset:
  cocktail += data
cocktail[:5]

['조주기능사 빈자리 신청 가능할려나...\n이게 신청기한이 은근 빡세네...\n애초에 이런거 안봐서 몰랐음\n그래서 빈자리 신청 기한도 있는데\n오픈하자 마자 신청하면 가능할려나?',
 '그냥 그렇네요\n담에 마가리타나 해볼까',
 '타오바오 고수들 질문좀\n결제 하려고 누르면 저렇게 문제있다고 뜨면서 결제대기페이지 가라고 함\n근데 가서 또 결제 누르면 결제할 수 없는 상품이 있다고 뜨네\n결제할 카드입력 하는곳도 없고 왜케어렵노',
 '칵갤 티키쟁이들아\n머해먹음',
 '조주기능 필기교재 득템\n살까말까 하다가 학교 도서관에 있길래 빌려옴 종강하면 공부해봐야지']

In [38]:
class CustomTokenizer:
  def __init__(self, tokenizer):
    self.tokenizer = tokenizer
  def __call__(self, target):
    return self.tokenizer.tokenize(target)

In [39]:
custom_tokenizer = CustomTokenizer(tokenizer)
vectorizer = CountVectorizer(tokenizer=custom_tokenizer, max_features=3000)

In [40]:
# bertopic_model = BERTopic(embedding_model=model,
#                  vectorizer_model=vectorizer,
#                  nr_topics=50,
#                  top_n_words=10,
#                  calculate_probabilities=True)
bertopic_model = BERTopic(language='multilingual',
                          nr_topics=50,
                          top_n_words=10,
                          calculate_probabilities=True)

In [41]:
topics, probs = bertopic_model.fit_transform(cocktail)

In [42]:
bertopic_model.visualize_topics()

In [43]:
bertopic_model.visualize_distribution(probs[0])

In [44]:
for i in range(0, 50):
  print(i,'번째 토픽 :', bertopic_model.get_topic(i))

0 번째 토픽 : [('맨하탄', 0.3958302882799244), ('맨해튼', 0.3225822697398207), ('나눔릴', 0.3221961797239609), ('받고', 0.16110278679565737), ('맨하튼', 0.14225507985158928), ('브루클린', 0.14225507985158928), ('브롱스', 0.1288086859525723), ('라이', 0.0958213765202444), ('맨해튼받고', 0.07785392896267686), ('리몬치노소다', 0.07785392896267686)]
1 번째 토픽 : [('이미지', 0.15662803538680145), ('순서', 0.15555446511692167), ('내릴내받', 0.04572722953203948), ('받고', 0.032875719891460015), ('사진', 0.023992108297347266), ('입고', 0.019852227499772685), ('데고르쥬망을', 0.019368313362211603), ('쿼캐', 0.01818639322823041), ('라프', 0.017737277576427044), ('동영상', 0.015155475559997427)]
2 번째 토픽 : [('얼음', 0.13040558288384002), ('탄산이', 0.05524238372611256), ('얼음을', 0.04811185942076244), ('얼음에', 0.042030367443717555), ('더블스트레인', 0.042030367443717555), ('스트레이너', 0.04028530765115875), ('얼린', 0.02966047211480505), ('라이프해커스', 0.02966047211480505), ('코블러사용', 0.02966047211480505), ('벽면에', 0.02966047211480505)]
3 번째 토픽 : [('얼음', 0.12097719801806006), ('크러쉬드', 0.076

In [45]:
for i in range(8):
  print(topics.count(i))

22
120
22
15
12
83
38
16


In [46]:
print(topics)

[-1, 36, 10, -1, -1, -1, 30, -1, 21, 13, -1, 37, -1, -1, 25, -1, -1, 18, 23, -1, -1, 10, 28, 9, 13, 24, -1, 9, 29, 9, 10, -1, 13, 13, -1, -1, -1, -1, 10, -1, 9, 18, 37, 13, -1, -1, -1, 10, 23, 9, 28, -1, 32, -1, 7, -1, 7, 10, 7, 9, 13, 30, -1, -1, 1, 9, -1, -1, -1, 31, 36, -1, 11, -1, 13, -1, 24, 9, 7, -1, -1, 20, 1, 9, 13, 23, 13, 35, -1, -1, -1, 9, 1, 9, -1, -1, 9, 13, 23, -1, -1, -1, 10, 9, 29, -1, -1, 2, -1, 18, -1, -1, 29, 28, -1, 9, 9, 9, 29, 9, 4, -1, -1, -1, 5, -1, -1, 35, 1, -1, 9, 10, 13, -1, -1, 1, -1, -1, 27, -1, 28, -1, 29, 1, -1, 37, -1, 18, 23, 13, 9, 1, -1, 29, 9, 25, -1, 9, -1, 9, 25, -1, -1, 9, -1, -1, 9, 9, -1, -1, 1, 36, -1, 5, -1, 37, -1, -1, 18, 9, 20, 29, -1, 13, -1, 29, -1, -1, 31, 13, 30, -1, -1, -1, 23, 1, 1, 9, 1, 1, -1, 20, 13, 13, 15, -1, 17, -1, -1, -1, 13, 1, -1, 36, -1, 28, 20, -1, 29, 13, -1, 9, 9, -1, -1, 0, 28, 9, 29, -1, 5, -1, 5, -1, 24, 1, 13, 9, -1, 9, -1, 34, 29, -1, 18, 10, 9, -1, 13, -1, 17, -1, -1, -1, -1, 27, 20, 9, 13, -1, 10, -1, 9, -1, 12,

In [47]:
from sklearn.cluster import KMeans

n_clusters = 4
kmeans = KMeans(n_clusters=n_clusters, random_state=0)
kmeans.fit(probs)
labels = kmeans.labels_


In [48]:
real_labels = []
for i in range(len(dataset)):
  real_labels += [i] * len(dataset[i])

from sklearn.metrics import confusion_matrix
print(confusion_matrix(real_labels, labels))


[[  27  301 2455   60]
 [   3   37  165    5]
 [   2   59  132    2]
 [  14  393  628   88]]


In [49]:
import numpy as np
from scipy.optimize import linear_sum_assignment
from sklearn.metrics import confusion_matrix

def calc_purity(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    _, col_ind = linear_sum_assignment(cm, maximize=True)
    reordered_cm = cm[:, col_ind]
    purity = np.sum(np.max(reordered_cm, axis=0)) / np.sum(reordered_cm)
    return purity, reordered_cm

purity, reordered_cm = calc_purity(real_labels, labels)

print("Purity: ", purity)
print("Reordered Confusion Matrix: \n", reordered_cm)

Purity:  0.67787691603752
Reordered Confusion Matrix: 
 [[2455   60   27  301]
 [ 165    5    3   37]
 [ 132    2    2   59]
 [ 628   88   14  393]]
